In [579]:
!pip install simpletransformers

In [580]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report, accuracy_score, f1_score
from tqdm import tqdm,trange

In [581]:
!pwd

/home/admin-user/swardi/ner-drone


In [582]:
dataset_train_path = './dataset/ner_drone_train_3(7030).csv'
dataset_test_path = './dataset/ner_drone_test_3(7030).csv'
dataset_train = pd.read_csv(dataset_train_path, encoding= 'unicode_escape')
dataset_test = pd.read_csv(dataset_test_path, encoding= 'unicode_escape')
dataset_train

,sentence_id,words,labels
0,1,Home,O
1,1,Point,O
2,1,recorded,O
3,2,GPS,B-FNC
4,2,Position,O
...,...,...,...
2655,469,recorded,O
2656,469,ft,O
2657,470,GPS,B-FNC
2658,470,Position,O


In [602]:
dataset_train["labels"].value_counts()

O        1954
B-FNC     229
B-ACT     103
B-CMP     102
I-ISS      60
B-STE      56
B-ISS      47
I-STE      35
I-FNC      30
I-ACT      25
I-CMP      19
Name: labels, dtype: int64

In [603]:
dataset_test["labels"].value_counts()

O        740
B-FNC    100
B-CMP     47
B-ACT     38
I-ISS     19
B-ISS     18
B-STE     17
I-ACT     11
I-FNC     10
I-STE     10
I-CMP      8
Name: labels, dtype: int64

In [583]:
label = dataset_train["labels"].unique().tolist()
label

['O',
 'B-FNC',
 'B-STE',
 'I-STE',
 'B-ACT',
 'B-ISS',
 'I-ISS',
 'B-CMP',
 'I-CMP',
 'I-FNC',
 'I-ACT']

In [584]:
from simpletransformers.ner import NERModel, NERArgs

args = NERArgs()
args.num_train_epochs = 4
args.learning_rate = 0.0001 # 0.0001  5e-5, 3e-5, 2e-5
args.overwrite_output_dir = True
args.train_batch_size = 16
args.eval_batch_size = 16

In [585]:
model = NERModel("bert", "bert-base-cased", labels=label, args =args, use_cuda=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [586]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [587]:
model.train_model(dataset_train, eval_data = dataset_test, acc=accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/30 [00:00<?, ?it/s]

(120, 0.24134966753190384)

In [588]:
result, model_outputs, preds_list = model.eval_model(dataset_test)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [589]:
result

{'eval_loss': 0.03633740302533484,
 'precision': 0.9464285714285714,
 'recall': 0.9636363636363636,
 'f1_score': 0.9549549549549549}

In [590]:
# Groupby and collect columns
dataset_test_group = dataset_test.groupby(['sentence_id'], as_index=False)['words', 'labels'].agg(lambda x: list(x))
y_test = dataset_test_group['labels']
y_test

/tmp/ipykernel_15342/1180599873.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataset_test_group = dataset_test.groupby(['sentence_id'], as_index=False)['words', 'labels'].agg(lambda x: list(x))


0                                         [B-ACT, I-ACT]
1                                         [B-ACT, I-ACT]
2                                 [O, O, O, B-ACT, O, O]
3                                [B-CMP, B-FNC, O, O, O]
4               [O, O, O, O, O, O, O, O, B-CMP, O, O, O]
                             ...                        
197                                   [O, O, O, O, O, O]
198                    [B-FNC, I-FNC, O, O, O, O, B-ACT]
199    [O, B-ISS, I-ISS, O, O, B-CMP, O, O, O, O, O, ...
200                                       [B-ACT, I-ACT]
201                                   [O, O, O, O, O, O]
Name: labels, Length: 202, dtype: object

In [591]:
y_pred_list = []
y_test_list = []
for row in range(0, len(preds_list)):
  y_pred_list = np.concatenate((y_pred_list, preds_list[row]), axis=0)
  y_test_list = np.concatenate((y_test_list, y_test[row]), axis=0)
y_pred_list

array(['B-ACT', 'I-ACT', 'B-ACT', ..., 'O', 'O', 'O'], dtype='<U32')

In [592]:
data_prediction = pd.DataFrame({'actual_class': y_test_list, 'predicted_class': y_pred_list})
data_prediction

,actual_class,predicted_class
0,B-ACT,B-ACT
1,I-ACT,I-ACT
2,B-ACT,B-ACT
3,I-ACT,I-ACT
4,O,O
...,...,...
1013,O,O
1014,O,O
1015,O,O
1016,O,O


In [593]:
confusion_matrix = pd.crosstab(data_prediction['predicted_class'], data_prediction['actual_class'])
confusion_matrix

actual_class,B-ACT,B-CMP,B-FNC,B-ISS,B-STE,I-ACT,I-CMP,I-FNC,I-ISS,I-STE,O
predicted_class,,,,,,,,,,,
B-ACT,38,0,0,0,0,0,0,0,0,0,0
B-CMP,0,44,0,0,0,0,0,0,0,0,1
B-FNC,0,3,100,0,0,0,0,0,0,0,1
B-ISS,0,0,0,15,0,0,0,0,0,0,0
B-STE,0,0,0,0,17,0,0,0,0,0,0
I-ACT,0,0,0,0,0,11,0,0,0,0,0
I-CMP,0,0,0,0,0,0,8,0,0,0,0
I-FNC,0,0,0,0,0,0,0,10,0,0,0
I-ISS,0,0,0,0,0,0,0,0,18,0,1


In [ ]:
confustion_matrix.to_excel('confusion_matris.csv')

In [594]:
y_test = data_prediction['actual_class'].to_list()
y_pred = data_prediction['predicted_class'].to_list()

In [595]:
from sklearn.metrics import classification_report
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, labels=['O', 'B-ACT', 'I-ACT', 'B-CMP', 'I-CMP', 'B-FNC', 'I-FNC', 'B-ISS', 'I-ISS', 'B-STE', 'I-STE'])).T

,precision,recall,f1-score,support
O,0.994587,0.993243,0.993915,740.000000
B-ACT,1.000000,1.000000,1.000000,38.000000
I-ACT,1.000000,1.000000,1.000000,11.000000
B-CMP,0.977778,0.936170,0.956522,47.000000
I-CMP,1.000000,1.000000,1.000000,8.000000
B-FNC,0.961538,1.000000,0.980392,100.000000
I-FNC,1.000000,1.000000,1.000000,10.000000
B-ISS,1.000000,0.833333,0.909091,18.000000
I-ISS,0.947368,0.947368,0.947368,19.000000
B-STE,1.000000,1.000000,1.000000,17.000000


In [596]:
from seqeval.metrics import classification_report
pd.DataFrame(classification_report([y_test], [y_pred], mode='strict', scheme='IOB2' output_dict=True)).T

,precision,recall,f1-score,support
ACT,1.000000,1.000000,1.000000,38.0
CMP,0.977778,0.936170,0.956522,47.0
FNC,0.961538,1.000000,0.980392,100.0
ISS,0.722222,0.722222,0.722222,18.0
STE,0.894737,1.000000,0.944444,17.0
micro avg,0.946429,0.963636,0.954955,220.0
macro avg,0.911255,0.931678,0.920716,220.0
weighted avg,0.946909,0.963636,0.954779,220.0


In [597]:
prediction, model_output = model.predict(["Aircraft is returning to the Home Point Minimum RTH Altitude is 30m You can reset the RTH Altitude in Remote Controller Settings after cancelling RTH"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [598]:
prediction

[[{'Aircraft': 'B-CMP'},
  {'is': 'O'},
  {'returning': 'O'},
  {'to': 'O'},
  {'the': 'O'},
  {'Home': 'O'},
  {'Point': 'O'},
  {'Minimum': 'O'},
  {'RTH': 'B-ACT'},
  {'Altitude': 'O'},
  {'is': 'O'},
  {'30m': 'O'},
  {'You': 'O'},
  {'can': 'O'},
  {'reset': 'O'},
  {'the': 'O'},
  {'RTH': 'B-ACT'},
  {'Altitude': 'O'},
  {'in': 'O'},
  {'Remote': 'O'},
  {'Controller': 'B-CMP'},
  {'Settings': 'B-FNC'},
  {'after': 'O'},
  {'cancelling': 'O'},
  {'RTH': 'B-ACT'}]]

In [599]:
prediction2, model_output2 = model.predict(["Weak signal Avoid blocking the antennas and keep the antennas parallel to and facing toward the aircraft during flight"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [600]:
prediction2

[[{'Weak': 'O'},
  {'signal': 'O'},
  {'Avoid': 'O'},
  {'blocking': 'O'},
  {'the': 'O'},
  {'antennas': 'O'},
  {'and': 'O'},
  {'keep': 'O'},
  {'the': 'O'},
  {'antennas': 'O'},
  {'parallel': 'O'},
  {'to': 'O'},
  {'and': 'O'},
  {'facing': 'O'},
  {'toward': 'O'},
  {'the': 'O'},
  {'aircraft': 'B-CMP'},
  {'during': 'O'},
  {'flight': 'O'}]]